In [413]:
import numpy as np
import scipy.stats as st
import math
import matplotlib.pyplot as plt

In [414]:
plt.rcParams.update({'legend.fontsize': 30, 'axes.labelsize' : 30, 'axes.titlesize': 20})

In [415]:
N = 10**3
TernConst = 1

In [416]:
def ternarnik(x, y, func):
    m1 = x + (y - x) / 3
    m2 = y - (y - x) / 3
#     print("m1 = {}, m2 = {}".format(m1, m2))
#     print("func(m1) = {}, func(m2) = {}".format(func(m1), func(m2)))
    if func(m1) < func(m2):
#         print("x = {}, y = {}".format(m1, y))
        return (m1, y)
    else:
#         print("x = {}, y = {}".format(x, m2))
        return (x, m2)
def do_ternar(x, y, func):
    for i in range(TernConst):
        (x, y) = ternarnik(x, y, func)
    return (x, y)

In [417]:
def make_random(n):
    P_base = list(np.random.rand(n - 1))
    P_base.append(0)
    P_base.append(1)
    P_base.sort()
    P = np.array([P_base[i] - P_base[i - 1] for i in range(1, n + 1)])
    P = np.floor(P * 100) / 100
    P[0] += 1 - np.sum(P)
    p = np.int64(np.random.rand(n, n) * 100)
    #p = (np.random.rand(n, n) - np.ones((n, n)) / 2) * 2
    return [P, p]

In [418]:
def base_func(P, p, x):
    x = np.array(x)
    p = np.array(p)
    result = 0
    for (Pi, pi) in zip (P, p):
#         print(Pi)
#         print(x)
#         print(pi)
#         print(pi @ x)
        result += Pi * math.log(pi @ x)
    return result
def base_grad(P, p, x):
    n = len(x)
    x = np.array(x)
    p = np.array(p)
    result = np.zeros(n)
    for (Pi, pi, p_j) in zip (P, p, p.transpose()):
        result += pi * (Pi / (pi @ x))
#     print(result)
    result -= np.ones((n)) * ((np.ones((n)) @ result) / n)
    return result

In [419]:
def make_func(P, p):
    return lambda x: base_func(P, p, x)
def make_grad(P, p):
    return lambda x: base_grad(P, p, x)

In [420]:
def next_grad_point_with_step(x_prev, lambd, grad):
    return(x_prev + lambd * grad(x_prev))
def next_grad_point(x_prev, grad, func):
    grad = grad(x_prev)
#     print(grad)
    t = None
    for (x, g) in zip(x_prev, grad):
        comp = None
        if g > 0:
            comp = (1 - x) / g
        elif g < 0:
            comp = -x / g
#         print('x = {}, grad = {}, t = {}'.format(x, g, comp))
        if (t == None) or comp < t:
            t = comp
#     print(x_prev + t * 98 / 100 * grad)
    points = do_ternar(x_prev, x_prev + t * 98 / 100 * grad, func)
    return((points[0] + points[1]) / 2)

In [421]:
config_base = [
    {
        'P': [0.5, 0.5],
        'p': [[1, 1], [1, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.5, 0.5],
        'p': [[1, 20], [1, 1]],
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.5, 0.5],
        'p': [[19, 20], [1, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.1, 0.9],
        'p': [[19, 1], [20, 1]], 
        'lambda': 1,
        'Graphs': []
    }
]
random_config = [3, 3, 3, 4, 4, 4, 4, 6, 6, 6, 10, 10, 20]

In [422]:
data = config_base.copy()
for k in random_config:
    ran = make_random(k)
    data.append({
        'P': ran[0],
        'p': ran[1],
        'lambda': 1,
        'Graphs': []
    })

In [423]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p)
    x0 = np.ones((len(P))) / len(P)
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    print(data_map)
    for i in range(N):
#         print("Grad: {}".format(grad(Xs[-1])))
        if np.linalg.norm(grad(Xs[-1])) < 0.01:
            print("Check! Grad\n")
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
#         print("Next x: {}".format(Xs[-1]))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < 10**-4):
            print("Check! Norm\n")
            break

{'P': [0.5, 0.5], 'p': [[1, 1], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}]}
Check! Grad

{'P': [0.5, 0.5], 'p': [[1, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.1756876285817388], 'Xs': [array([0.5, 0.5])]}]}
Check! Norm

{'P': [0.5, 0.5], 'p': [[19, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.4852072327848505], 'Xs': [array([0.5, 0.5])]}]}
Check! Norm

{'P': [0.1, 0.9], 'p': [[19, 1], [20, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [2.3464962407465344], 'Xs': [array([0.5, 0.5])]}]}
Check! Norm

{'P': array([0.32, 0.2 , 0.48]), 'p': array([[ 5, 66, 45],
       [ 1, 23, 75],
       [67, 83, 59]]), 'lambda': 1, 'Graphs': [{'x0': array([0.33333333, 0.33333333, 0.33333333]), 'values': [3.9058809834559542], 'Xs': [array([0.33333333, 0.33333333, 0.33333333])]}]}
Check! Norm

{'P': array([0.55, 0.09, 0.36]), 'p': array([[33, 38, 27],
       [25, 

In [424]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p) 
    x0 = np.zeros(len(P))
    x0[0] = 0.99
    x0[1] = 0.01
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    print(data_map)
    for i in range(N):
#         print("Grad: {}".format(grad(Xs[-1])))
        if np.linalg.norm(grad(Xs[-1])) < 0.01:
            print("Check! Grad\n")
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
#         print("Next x: {}".format(Xs[-1]))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < 10**-4):
            print("Check! Norm\n")
            break

{'P': [0.5, 0.5], 'p': [[1, 1], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}, {'x0': array([0.99, 0.01]), 'values': [0.0], 'Xs': [array([0.99, 0.01])]}]}
Check! Grad

{'P': [0.5, 0.5], 'p': [[1, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.1756876285817388, 1.407903920679879, 1.4684768622370055, 1.4878722847883117, 1.4944241759814736, 1.4966756050444372, 1.497453740049126, 1.4977232110939578, 1.4978165938333992, 1.4978489624457942], 'Xs': [array([0.5, 0.5]), array([0.17333333, 0.82666667]), array([0.06008889, 0.93991111]), array([0.02083081, 0.97916919]), array([0.00722135, 0.99277865]), array([0.0025034, 0.9974966]), array([8.67845692e-04, 9.99132154e-01]), array([3.00853173e-04, 9.99699147e-01]), array([1.04295767e-04, 9.99895704e-01]), array([3.61558658e-05, 9.99963844e-01])]}, {'x0': array([0.99, 0.01]), 'values': [0.08697665356171899], 'Xs': [array([0.99, 0.01])]}]}
Check! Norm

{'P': [0.5, 0

In [425]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p) 
    x0 = np.zeros(len(P))
    x0[0] = 0.5
    x0[1] = 0.5
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    print(data_map)
    for i in range(N):
#         print("Grad: {}".format(grad(Xs[-1])))
        if np.linalg.norm(grad(Xs[-1])) < 0.01:
            print("Check! Grad\n")
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
#         print("Next x: {}".format(Xs[-1]))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < 10**-4):
            print("Check! Norm\n")
            break

{'P': [0.5, 0.5], 'p': [[1, 1], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}, {'x0': array([0.99, 0.01]), 'values': [0.0], 'Xs': [array([0.99, 0.01])]}, {'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}]}
Check! Grad

{'P': [0.5, 0.5], 'p': [[1, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.1756876285817388, 1.407903920679879, 1.4684768622370055, 1.4878722847883117, 1.4944241759814736, 1.4966756050444372, 1.497453740049126, 1.4977232110939578, 1.4978165938333992, 1.4978489624457942], 'Xs': [array([0.5, 0.5]), array([0.17333333, 0.82666667]), array([0.06008889, 0.93991111]), array([0.02083081, 0.97916919]), array([0.00722135, 0.99277865]), array([0.0025034, 0.9974966]), array([8.67845692e-04, 9.99132154e-01]), array([3.00853173e-04, 9.99699147e-01]), array([1.04295767e-04, 9.99895704e-01]), array([3.61558658e-05, 9.99963844e-01])]}, {'x0': array([0.99, 0.01]), 'values': [0.086976

       [40, 49, 82, 82]]), 'lambda': 1, 'Graphs': [{'x0': array([0.25, 0.25, 0.25, 0.25]), 'values': [3.9309191484600152, 4.101267387917515, 4.1638864467380845, 4.18398436377035, 4.190674128019992, 4.192961499540354, 4.19375069951291, 4.194023839908544, 4.194118474726461, 4.194151274995547], 'Xs': [array([0.25, 0.25, 0.25, 0.25]), array([0.25705266, 0.11289969, 0.54338098, 0.08666667]), array([0.24995522, 0.04033181, 0.67966853, 0.03004444]), array([0.24675856, 0.01398169, 0.72690141, 0.01235834]), array([0.24561812, 0.00484699, 0.74301776, 0.00651713]), array([0.24522055, 0.00168029, 0.74857437, 0.0045248 ]), array([2.45082520e-01, 5.82500197e-04, 7.50497021e-01, 3.83795962e-03]), array([2.45034649e-01, 2.01933401e-04, 7.51163104e-01, 3.60031334e-03]), array([2.45018051e-01, 7.00035792e-05, 7.51393961e-01, 3.51798417e-03]), array([2.45012297e-01, 2.42679074e-05, 7.51473985e-01, 3.48944998e-03])]}, {'x0': array([0.99, 0.01, 0.  , 0.  ]), 'values': [3.8819691352705026, 3.881969135270502

In [426]:
rows_per_data = 2 + (len(data[0]['Graphs']) // 3 + 1) * 2 + 1
gridsize = (len(data) * rows_per_data, 3)
fig = plt.figure(figsize=(40, 40 * len(data)), facecolor=matplotlib.colors.get_named_colors_mapping()['xkcd:cream'])
plt.subplots_adjust(hspace=0.8)
for i in range(len(data)):
    data_map = data[i]
    P = data_map['P']
    p = data_map['p']
    labels = ['$x_{' + '{}'.format(i + 1) + '}$' for i in range(len(P))]
    ax0 = plt.subplot2grid(gridsize, (i * rows_per_data, 0), colspan=3, rowspan=2)
    for (Pi, pi) in zip(P, p):
        params += r'$\pi$: {:.2f}, $p_j$: {}'.format(Pi, pi) + '\n'
    ax0.set_title(params, {'fontsize': 30}, loc='left')
    ax0.set_xlabel(r'$k$')
    ax0.set_ylabel(r'$F(x_k)$')
    graphs = data_map['Graphs']
    for j in range(len(gaphs)):
        graph = graphs[j]
        Xs = np.array(graph['Xs'])
        x0 = graph['x0']
        n = len(Xs)
        graph_x = range(n)
        ax0.plot(graph_x, graph['values'], label='$x_0 = ${}'.format(np.floor(x0*100) / 100))
        axi = plt.subplot2grid(gridsize, (i * rows_per_data + (j // 3) * 2 + 2, j % 3), colspan=1, rowspan=2)
        params = ''
        axi.stackplot(graph_x, *[Xs[:, i] for i in range(len(P))], labels=labels)
        axi.legend()
        axi.set_title('$x_0 = ${}'.format(np.floor(x0*100) / 100))
        axi.set_xlabel(r'$k$')
        axi.set_ylabel(r'$X$')
    ax0.legend()

In [427]:
import pprint
pprint.pprint(data)

[{'Graphs': [{'Xs': [array([0.5, 0.5])],
              'values': [0.0],
              'x0': array([0.5, 0.5])},
             {'Xs': [array([0.99, 0.01])],
              'values': [0.0],
              'x0': array([0.99, 0.01])},
             {'Xs': [array([0.5, 0.5])],
              'values': [0.0],
              'x0': array([0.5, 0.5])}],
  'P': [0.5, 0.5],
  'lambda': 1,
  'p': [[1, 1], [1, 1]]},
 {'Graphs': [{'Xs': [array([0.5, 0.5]),
                     array([0.17333333, 0.82666667]),
                     array([0.06008889, 0.93991111]),
                     array([0.02083081, 0.97916919]),
                     array([0.00722135, 0.99277865]),
                     array([0.0025034, 0.9974966]),
                     array([8.67845692e-04, 9.99132154e-01]),
                     array([3.00853173e-04, 9.99699147e-01]),
                     array([1.04295767e-04, 9.99895704e-01]),
                     array([3.61558658e-05, 9.99963844e-01])],
              'values': [1.175687628581738

  'P': array([0.03, 0.37, 0.11, 0.17, 0.  , 0.32]),
  'lambda': 1,
  'p': array([[79, 51, 92, 48, 72, 50],
       [60, 49, 44, 32, 98, 88],
       [53, 74, 57, 61, 57,  6],
       [30, 72, 65, 52, 88, 12],
       [35, 57, 33, 77,  2, 52],
       [38, 83, 26, 75, 70, 66]])},
 {'Graphs': [{'Xs': [array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]),
                     array([0.05777778, 0.12246306, 0.29978933, 0.27623846, 0.07191856,
       0.17181281]),
                     array([0.02002963, 0.10256041, 0.34549445, 0.31068332, 0.03249786,
       0.18873434]),
                     array([0.0069436 , 0.09495219, 0.36118267, 0.32238039, 0.01780872,
       0.19673242]),
                     array([0.00240712, 0.09222578, 0.3666009 , 0.32640791, 0.01258542,
       0.19977287]),
                     array([0.00083447, 0.09126984, 0.36847674, 0.32780083, 0.01075864,
       0.20085949]),
                     array([2.89281897e-04, 9.09371399e-02, 3.69126736e

In [433]:
grad([0.97, 0.03, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

array([-2.22942269,  0.23260946,  0.50728063,  0.66753654,  1.370249  ,
       -1.16314014, -0.95363493, -1.35296875, -0.23452164,  0.66758241,
        0.16494026,  2.55870307,  0.51979448, -0.24530641, -0.34644102,
       -0.2773186 , -0.04389814,  0.65548553,  1.06755696, -1.56508603])